In [1]:
import numpy as np
import torch
import torch.nn as nn

import models.BERT as model
from utils import DataConfig

np.random.seed(3407)
torch.manual_seed(3407)
torch.cuda.manual_seed_all(3407)
torch.backends.cudnn.deterministic = True  # 保证每次结果一样
torch.backends.cudnn.benchmark = False
UNK, PAD = '<UNK>', '<PAD>'  # 未知字，padding符号

In [2]:
model_config = model.ModelConfig()

In [3]:
from utils import BertDataConfig, BertDataset

data_config = BertDataConfig()
val_dataset = BertDataset(data_config, data_class='val')

'蹦預蕊颛紺凛宝姝蕊绳蹦基預姝宝凛凛蕊绳虾仰蹦陌基西使姝咎蕊使'

In [ ]:
class Model(nn.Module):
    def __init__(self, model_config, data_config):
        super(Model, self).__init__()
        # 加载预训练模型
        self.pretrained = BertModel.from_pretrained('bert-base-chinese')
        self.fc = torch.nn.Linear(768, data_config.num_classes)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = self.pretrained(input_ids=input_ids,
                                  attention_mask=attention_mask,
                                  token_type_ids=token_type_ids)
        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out


In [ ]:
model = Model(model_config, data_config).to(data_config.device)

In [ ]:
model(torch.randint(1, 10, [2, 30]).to(data_config.device)).size()

In [ ]:
def init_network(model, method='xavier', exclude='embedding'):
    for name, w in model.named_parameters():
        if exclude not in name:  # 如果不是嵌入层
            if 'weight' in name:  # weight 三种初始化方式
                if method == 'xavier' and len(w.size() < 2):
                    nn.init.xavier_normal_(w)
                elif method == 'kaiming':
                    nn.init.kaiming_normal_(w)
                else:
                    nn.init.normal_(w)
            elif 'bias' in name:  # bias 置0
                nn.init.constant_(w, 0)
            else:
                pass

In [ ]:
init_network(model)

In [ ]:
from torchinfo import summary

summary(model, input_size=(1, 30), dtypes=[torch.long])

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 读取CSV文件
df = pd.read_csv('./ship_data/experiment_data.csv')  # 替换为你的CSV文件路径

# 划分数据集
train_df, temp_df = train_test_split(df, test_size=0.1, random_state=3407)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=3407)

# 保存划分后的数据集为新的CSV文件
train_df.to_csv('train_dataset.csv', index=False)
val_df.to_csv('val_dataset.csv', index=False)
test_df.to_csv('test_dataset.csv', index=False)


In [ ]:
import torch
import torch.nn.functional as F

# 假设你的输入是一个大小为[2, 30, 100]的tensor
input_tensor = torch.randn(2, 30, 100)

# 执行最大池化操作，保留最大的两个值
output_tensor = F.max_pool1d(input_tensor, kernel_size=2, stride=1)

print(output_tensor.shape)  # 输出应为[2, 2, 100]


In [ ]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files={"train": "./ship_data/train_dataset.csv", "test": "./ship_data/test_dataset.csv","val":"./ship_data/val_dataset.csv"})

In [ ]:
dataset['train'][0]

In [ ]:
from transformers import AutoModelForSequenceClassification

model=AutoModelForSequenceClassification.from_pretrained("bert-base-chinese",num_labels=5)

In [ ]:
all_layers = list(model.children())

In [ ]:
all_layers

In [ ]:
from torchinfo import summary
import torch
batch_size=1
summary(model, input_size=(batch_size, 30),dtypes=[torch.long])

In [ ]:
type(model)

In [ ]:
from transformers import BertModel

#加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese')

In [11]:
from transformers import BertTokenizer

#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')
out=token.encode('今天是个好日子')
token.decode(out)

'[CLS] 今 天 是 个 好 日 子 [SEP]'

In [33]:
zidian=token.get_vocab()